# Integrating Flotorch Session Management with LangGraph

This notebook demonstrates how to build a LangGraph agent integrated with Flotorch's session management using `FlotorchLanggraphSession` as the checkpointer. The agent persists conversation history within a session, enabling context-aware interactions, and is equipped with a custom tools.

### Prerequisites
Configure your model, API key, and session details in the Flotorch console (https://console.flotorch.cloud/).

### Viewing Logs
Logs can be viewed in the logs tab of the Flotorch console (https://console.flotorch.cloud/).

### Key Concepts:
- **`FlotorchLangChainLLM`**: The language model driving the agent's reasoning capabilities.
- **`FlotorchLanggraphSession`**: The session storage mechanism for persisting conversation history in LangGraph.
- **`create_react_agent`**: LangGraph's function to create a ReAct-style agent with tools and state persistence.
- **Custom Tool**: `multiply` and `addition` tools to perform arithmetic addition and multiplication integrated with the agent.

## 1. Setup and Configuration

The following cells install required packages, configure API credentials, and import necessary components from the Flotorch and LangGraph libraries, assuming they are available in the project directory.

In [ ]:
%pip install flotorch[langgraph]

In [ ]:
FLOTORCH_API_KEY = "<YOUR FLOTORCH_API_KEY>"
FLOTORCH_BASE_URL = "<YOUR FLOTORCH_BASE_URL>"  
FLOTORCH_MODEL_ID = "<YOUR FLOTORCH_MODEL_ID>"
USER_ID = "YOUR USER_ID" #eg : flotorch_user500
APP_ID = "YOUR APP_ID"   #eg : flotorch_app500

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from flotorch.langchain.llm import FlotorchLangChainLLM
from flotorch.langgraph.sessions import FlotorchLanggraphSession

print("✔ Imported necessary libraries and loaded environment variables successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define a single custom tool using the `@tool` decorator from LangChain. The `multiply` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their product. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.

In [ ]:
@tool
def multiply(numbers: str) -> int:
    """
    Multiply two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The product of the two integers
    """
    print("Multiplying... custom tool")
    a, b = map(int, numbers.split(","))
    return a * b

print("✔ Custom multiplication tool defined successfully.")

## 3. Defining a Custom Tool

To extend the agent's capabilities, we define a custom tool using the `@tool` decorator from LangChain. The `addition` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their sum. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.

In [ ]:
@tool
def addition(numbers: str) -> int:
    """
    Addition of two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The Addition of the two integers
    """
    print("Adding... custom tool")
    a, b = map(int, numbers.split(","))
    return a + b

tools = [multiply,addition]

print("✔ Custom Addition tool defined successfully.")

## 4. Model Configuration

We initialize the `FlotorchLangChainLLM` with the Flotorch model configuration from the custom `llm` module. This serves as the reasoning engine for the LangGraph agent, enabling it to process user inputs and invoke the custom tool.

In [ ]:
model = FlotorchLangChainLLM(
    model_id=FLOTORCH_MODEL_ID,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

print(f"✔ Flotorch LLM model configured")

## 5. Session Storage Setup

We configure the `FlotorchLanggraphSession` as the checkpointer for LangGraph, enabling persistent storage of conversation history within a session. The session is uniquely identified by `app_name`, `user_id`, and `thread_id`.

In [ ]:
checkpointer = FlotorchLanggraphSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    app_name=APP_ID,
    user_id=USER_ID
)

config = {"configurable": {"thread_id": "flotorch-test-thread-500"}}

print(f"✔ Session storage initialized with thread ID: {config['configurable']['thread_id']}")

## 6. Agent Configuration

We create a LangGraph agent using `create_react_agent`, integrating the Flotorch LLM, the `multiply` and `addition` tools, and the `FlotorchLanggraphSession` checkpointer. The agent is designed to perform arithmetic addition and multiplication using the tools and maintain conversational context through session persistence.

In [ ]:
agent = create_react_agent(
    model=model,
    tools=tools,
    checkpointer=checkpointer
)

print("✔ LangGraph agent created successfully with session persistence.")

## 7. Interactive Chat

This interactive loop enables users to input queries, which the agent processes using the Flotorch-backed session storage to maintain context across the session. The agent invokes the `multiply` and `addition` tools for inputs requesting multiply(e.g., 'mul 3,4), addition (e.g., 'Add 3,4') and returns the raw numerical result. Type 'exit' to end the session.

In [ ]:
while True:
    user_query = input("You: ")
    if user_query.lower().strip() == "exit":
        break
    
    try:
        response = agent.invoke({"messages": user_query}, config)
        bot_reply = response["messages"][-1].content
        print(f"Assistant: {bot_reply}\n")
    except Exception as e:
        print(f"Error: {e}\n")

print("✔ Interactive session ended.")

## Summary

This notebook successfully demonstrated the creation of a LangGraph agent with session persistence using Flotorch's session management. By leveraging `FlotorchLanggraphSession` as the checkpointer, the agent maintains conversation context within a session, enhancing response relevance.

### Key Achievements

- **Session Persistence**  
  Configured `FlotorchLanggraphSession` as the checkpointer, enabling persistent storage of conversation history using `app_name`, `user_id`, and `thread_id`.

- **Tool Integration**  
  Implemented a custom tools `multiply` and `addition` tools to compute the  product of two integers and sum of two integers, ensuring focused and precise functionality.

- **Streamlined Integration**  
  Demonstrated a seamless integration of Flotorch's session management with LangGraph’s robust agent framework, enabling stateful and tool-augmented interactions.

## Summary

This notebook successfully demonstrated the creation of a LangGraph agent with session persistence using Flotorch's session management. By leveraging `FlotorchLanggraphSession` as the checkpointer, the agent maintains conversation context within a session, enhancing response relevance.

### Key Achievements

- **Session Persistence**  
  Configured `FlotorchLanggraphSession` as the checkpointer, enabling persistent storage of conversation history using `app_name`, `user_id`, and `thread_id`.

- **Tool Integration**  
  Implemented a custom tools `multiply` and `addition` tools to compute the  product of two integers and sum of two integers, ensuring focused and precise functionality.

- **Streamlined Integration**  
  Demonstrated a seamless integration of Flotorch's session management with LangGraph’s robust agent framework, enabling stateful and tool-augmented interactions.